In [9]:
import astropy.constants as ac
import astropy.units as au
import numpy as np

In [ ]:
# print the Bohr radius in Angström
aFG = (((3*np.pi**2)**(2/3)/5)*ac.hbar**2/ac.m_e).to(au.angstrom**5*au.GPa)
print(aFG)
print((ac.a0.to(au.angstrom))**5*56.32*au.TPa)
print(((1*au.cm)**3).to(au.angstrom**3)/1e24)

2*11.266*6.022e23/1e24

2336.902835838171 Angstrom5 GPa
2.3370516516897712 Angstrom5 TPa
0.9999999999999998 Angstrom3


13.5687704

In [28]:
def min_type(crys,a=None,c=None,d=None):

    # bcc Fe-15Si case
    if crys == 'bcc':
        a = d * np.sqrt(2)
        XFe = (85/MFe)/(85/MFe + 15/MSi)   # Molar fraction Fe-7Si
        XSi = 1.0 - XFe  # Molar fraction Fe-7Si
        V0 = 11.266*2
        KT0 = 227.9 # +-24.0
        KTP = 4.74
        Z = XFe*26.0 + XSi*14 # average Atomic number

    # hcp Fe-7Si case
    elif crys == 'hcp':
        XFe = (93/MFe)/(93/MFe + 7/MSi) 
        XSi = 1.0 - XFe
        V0 = 11.658*6 # A^3
        KT0 = 136.2 # +-20.0 GPa
        KTP = 5.97 # +-0.47
        Z = XFe*26.0 + XSi*14

    else:
        raise ValueError("Invalid crystal structure. Must be 'bcc' or 'hcp'.")
    
    return [V0, KT0, KTP, Z, XFe, XSi]

In [1]:
MFe = 55.845  # Molar mass of iron [g/mol]
MSi = 28.0855  # Molar mass of silicon [g/mol]
wFe = 0.85
wSi = 0.15
XSi = wSi / MSi / (wFe / MFe + wSi / MSi) # molar fraction of Si in Fe-15Si
print(XSi)

def alloy_conc(wSi):
    X2 =  wSi / XSi * MFe / (MFe*wSi + MSi*(1-wSi))
    X1 = 1 - X2
    return X1, X2

print(alloy_conc(0.00001))

0.25974881722697385
(0.999923450164077, 7.654983592298387e-05)


In [2]:
rtc = 3041843.3227286106
rc = 60.0+rtc
rb = 24905.865608419765+rc+rtc
km = 9.9273790821881729 
rhom = 5536.0  
rhoc = 11546.0
Hc = 1.0e-12
Hm = 2.0e-12
kc = 125.0
Tb = 4173.3167184879076 

Ttc = 4173.4654945261318

# Placeholder values
c1m = 0.0
c0m = 0.0
c1c = 0.0
c0c = 0.0

# dq/dr + 2q/r = pho*H --> T = -rho*H/(6*k) + c1/r + c0

# Construct matrix A
n_la = 5  # Define n_la based on your specific use case
A = np.array([
    [1.0, 1.0/rb, 0.0, 0.0, 0.0],
    [0.0, 1.0, 0.0, 0.0, -rc*rb/(rb-rc)],
    [0.0, 0.0, 1.0, 1.0/rc, -1.0],
    [0.0, 0.0, 0.0, 1.0, rtc*rc/(rc-rtc)],
    [0.0, 0.0, 0.0, 0.0, 1.0]
])

# Construct vector b
b = np.zeros(n_la)
b[0] = Tb + rhom*Hm/(6.0*km)*rb**2
b[1] = Tb*rb*rc/(rb-rc)-rhom*Hm/(6.0*km)*(rc**2*rb+rc*rb**2)
b[2] = rhoc*Hc/(6.0*kc)*rc**2
b[3] = Ttc*rtc*rc/(rc-rtc)-rhoc*Hc/(6.0*kc)*(rc**2*rtc+rc*rtc**2)
b[4] = (Ttc+Tb*km/kc*(rc-rtc)/(rb-rc))/(1+km/kc*(rc-rtc)/(rb-rc))

# Solve the linear system A * x = b
try:
    x = np.linalg.solve(A, b)
except np.linalg.LinAlgError:
    print("Error: The solution could not be computed.")
    exit()

# Extract the results
c1m = x[0]
c0m = x[1]
c1c = x[2]
c0c = x[3]
Tc = x[4]  # New temperature at the core-mantle boundary

#initially: Tc = 4172.9766230460846
print(f"Found Tc: {Tc} K")
print(Ttc-Tc)

NameError: name 'np' is not defined

In [ ]:
prof = 'M1_Fe3'
T = planets[prof]['T']
r = planets[prof]['r']
rho = planets[prof]['rho']
Cp = planets[prof]['Cp']
r = r[::-1]
Tc = planets[prof]['T'][0]
dr = r[0]-r[1]
n = len(r)-1

rho_c = rho[n]
Cp_c = Cp[n]

dTcdt = -5 * 1.0e-16

Vc = 4/3*np.pi*r[0]**3

QS1 = Vc * rho_c * Cp_c * sp.integrate.simpson(T,x=r,dx=dr)/Tc * 1.0e-12 * dTcdt
QS2 = 4*np.pi*sp.integrate.simpson(Cp*rho*T*r**2,x=r,dx=dr)/Tc * 1.0e-12 * dTcdt

print(len(r))
print(r[0])
print(f"QS1: {QS1}")
print(f"QS2: {QS2}")
print(f"QS1-QS2: {round((QS1-QS2)/QS1*100,2)}%")

qad = 125 * planets[prof]['alpha'][n] * planets[prof]['g'][n] / planets[prof]['Cp'][n] * planets[prof]['T'][n]
print(f"q_ad: {qad}")